In [16]:
import pandas as pd
import ast

In [17]:
def eda_csv(file_path):
    """
    Perform basic EDA on a CSV file.
    
    Parameters:
        file_path (str): Path to the CSV file.
        
    Returns:
        None
    """
    # 1. Load CSV
    df = pd.read_csv(file_path, na_values=['', 'NA', 'N/A', '-'])
    
    print("="*50)
    print("FIRST 5 ROWS")
    print(df.head())
    print("="*50)
    
    # 2. Shape of the dataframe
    print(f"Number of rows: {df.shape[0]}, Number of columns: {df.shape[1]}")
    print("="*50)
    
    # 3. Missing values
    print("MISSING VALUES PER COLUMN")
    print(df.isnull().sum())
    print("="*50)
    
    # 4. Number of duplicate rows
    num_duplicates = df.duplicated().sum()
    print(f"NUMBER OF DUPLICATE ROWS: {num_duplicates}")
    print("="*50)
    
    print("EDA COMPLETED")


In [18]:
df_movies_raw = pd.read_csv('Data/Raw/Movies.csv')

In [19]:
df_ratings_raw = pd.read_csv('Data/Raw/Ratings.csv')

In [20]:
df_keywords_raw = pd.read_csv("Data/Raw/Keywords.csv")

In [21]:
def parse_keywords(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

df_keywords_raw["keywords"] = df_keywords_raw["keywords"].apply(parse_keywords)
df_keywords_non_empty = df_keywords_raw[
    df_keywords_raw["keywords"].apply(len) > 0
]

df_movies_raw["genres"] = df_movies_raw["genres"].apply(parse_keywords)
df_movies_non_empty = df_movies_raw[
    df_movies_raw["genres"].apply(len) > 0
]


valid_movie_ids = set(df_keywords_non_empty["id"])

In [22]:
def parse_genres(x):
    if isinstance(x, list):      # already parsed
        return x
    if isinstance(x, str):       # string representation
        return ast.literal_eval(x)
    return []                    # NaN or anything else

print(f"Number of movies before removing empty genres: {df_movies_raw.shape[0]}")

df_movies_raw["genres_parsed"] = df_movies_raw["genres"].apply(parse_genres)

# Drop rows where genres list is empty
df_movies_raw = df_movies_raw[df_movies_raw["genres_parsed"].map(len) > 0]

print(f"Number of movies after removing empty genres: {df_movies_raw.shape[0]}")

Number of movies before removing empty genres: 45466
Number of movies after removing empty genres: 43024


In [23]:
# Data Cleaning and Sampling for Movies Dataset

# 1. Drop rows with any missing values
df_movies_raw_clean= df_movies_raw.dropna()

# 2. Drop duplicate rows (keep first occurrence)
df_movies_raw_unique = df_movies_raw_clean.drop_duplicates(subset=["id"])

df_movies_raw_unique["id"] = (
    df_movies_raw_unique["id"]
    .astype(str)
    .str.strip()
    .astype(int)
)

df_movies_filtered = df_movies_raw_unique[
    df_movies_raw_unique["id"].isin(valid_movie_ids)
]

print(f"Number of movies after filtering: {df_movies_filtered.shape[0]}")

# 3. Randomly sample 10,000 rows
df_movies_processed = df_movies_filtered.sample(n=10_000, random_state=42)

# (Optional) Save to new CSV
df_movies_processed.to_csv("Data/Processed/Movies.csv", index=False)

Number of movies after filtering: 30478


/tmp/ipykernel_75815/3960214948.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_raw_unique["id"] = (


In [24]:
eda_csv("Data/Processed/Movies.csv")

FIRST 5 ROWS
                                              genres      id  \
0  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  279828   
1                [{'id': 99, 'name': 'Documentary'}]  359871   
2  [{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...   18041   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...  188468   
4  [{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...   19067   

                                            overview  \
0  Set in a world where love is deemed illegal an...   
1  Following a long fascination with the religion...   
2  On a beautiful college campus, something ugly ...   
3  A country girl goes to the city and gets a job...   
4  In 1800, as Napoleon Bonaparte rises to power ...   

                        poster_path release_date  runtime  \
0  /u9FgFBuCdNxYZHrxgo0I03F9Fsq.jpg   2014-06-20     43.0   
1  /mAe9TuWsboDF25iFKMawChXUB9H.jpg   2016-05-05     99.0   
2  /jsSDDvXM1Lmxe8mIkCRRG8u9Q5k.jpg   2000-04-21     90.0   
3   /XNZt4hyABOsZ2NPi

In [25]:
df_movies_processed = pd.read_csv("Data/Processed/Movies.csv")

In [26]:
# Filtering Keywords Dataset Based on Processed Movies
# Get unique movie IDs
movie_ids = set(df_movies_processed["id"].unique())

# Filter keywords by movie IDs
filtered_keywords = df_keywords_raw[df_keywords_raw["id"].isin(movie_ids)]

# Remove duplicate rows by id
filtered_keywords = filtered_keywords.drop_duplicates(subset=["id"])

# Save filtered keywords to new CSV
filtered_keywords.to_csv("Data/Processed/Keywords.csv", index=False)

In [27]:
eda_csv("Data/Processed/Keywords.csv")

FIRST 5 ROWS
      id                                           keywords
0  31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
1  11860  [{'id': 90, 'name': 'paris'}, {'id': 380, 'nam...
2   9091  [{'id': 949, 'name': 'terrorist'}, {'id': 1562...
3   9273  [{'id': 409, 'name': 'africa'}, {'id': 1551, '...
4    451  [{'id': 30, 'name': 'individual'}, {'id': 549,...
Number of rows: 10000, Number of columns: 2
MISSING VALUES PER COLUMN
id          0
keywords    0
dtype: int64
NUMBER OF DUPLICATE ROWS: 0
EDA COMPLETED


In [28]:
# Filtering Ratings Dataset Based on Processed Movies
# Filter ratings by movie IDs
filtered_ratings = df_ratings_raw[df_ratings_raw["movieId"].isin(movie_ids)]

# Remove duplicate rows by movieId and userId
filtered_ratings = filtered_ratings.drop_duplicates(subset=["movieId", "userId"])

# Save filtered ratings to new CSV
filtered_ratings.to_csv("Data/Processed/Ratings.csv", index=False)

In [29]:
# If reading from a real file, use: df = pd.read_csv('your_file.csv')
df = pd.read_csv("Data/Processed/Ratings.csv")

# 2. Convert the timestamp to a readable datetime format
# unit='s' tells pandas these are seconds (Unix timestamp)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.to_csv("Data/Processed/Ratings.csv", index=False)

In [30]:
eda_csv("Data/Processed/Ratings.csv")

FIRST 5 ROWS
   userId  movieId  rating            timestamp
0       1      110     1.0  2015-03-09 22:52:09
1       2      605     4.0  1997-06-23 04:26:13
2       2      786     1.0  1997-06-23 04:14:09
3       2     1552     2.0  1997-06-23 04:38:55
4       3      500     2.0  2003-03-19 12:29:05
Number of rows: 3552697, Number of columns: 4
MISSING VALUES PER COLUMN
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
NUMBER OF DUPLICATE ROWS: 0
EDA COMPLETED
